In [2]:
import cv2
import sklearn
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import glob
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from matplotlib.image import NonUniformImage
from google.colab.patches import cv2_imshow


In [3]:
def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def get_pixel_else_0(l, idx, idy):
    if idx < int(len(l)) - 1 and idy < len(l[0]):
        return l[idx,idy]
    else:
        return 0

In [6]:
def get_pixel_sign(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value


In [7]:
def get_pixel_magn_diff(img, center, x, y):
    new_value = 0
    try:
        new_value = abs(int(img[x][y]) - int(center))
    except:
        pass
    return new_value


In [8]:

def lbp_calculated_pixel(img, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel_sign(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel_sign(img, center, x, y+1))       # right
    val_ar.append(get_pixel_sign(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel_sign(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel_sign(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel_sign(img, center, x, y-1))       # left
    val_ar.append(get_pixel_sign(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel_sign(img, center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val    


In [9]:

def clbp_sign_vec(img, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel_sign(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel_sign(img, center, x, y+1))       # right
    val_ar.append(get_pixel_sign(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel_sign(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel_sign(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel_sign(img, center, x, y-1))       # left
    val_ar.append(get_pixel_sign(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel_sign(img, center, x-1, y))       # top

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    # print(val_ar)
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val     


In [10]:

def clbp_Mag_vec(img, x, y):  
    center = img[x][y]
    # print("center = ",center)

    val_ar = []
    # print(img[x-1][y+1],"\n")
    val_ar.append(get_pixel_magn_diff(img, center, x-1, y+1))     # top_right
    # print("center = ",center)
    # print(img[x-1][y+1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x, y+1))       # right
    # print("center = ",center)
    # print(img[x][y+1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x+1, y+1))     # bottom_right
    # print("center = ",center)
    # print(img[x+1][y+1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x+1, y))       # bottom
    # print("center = ",center)
    # print(img[x+1][y],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x+1, y-1))     # bottom_left
    # print("center = ",center)
    # print(img[x+1][y-1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x, y-1))       # left
    # print("center = ",center)
    # print(img[x][y-1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x-1, y-1))     # top_left
    # print("center = ",center)
    # print(img[x-1][y-1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x-1, y))       # top
    # print("center = ",center)
    # print(img[x-1][y],"\n")

    return val_ar    


In [11]:
def t_paper(x,c):
    if(x>=c):
        return 1
    return 0

In [12]:
def clbp_S_Operator(img,x,y):
    return clbp_sign_vec(img,x,y)

In [13]:
def clbp_M_Operator(mp,P):
    c_threshold = np.mean(mp)
    tp = np.where(mp >= c_threshold, 1, 0)
    pp2 = (np.array(list(range(0,P))))
    return np.dot(tp,pp2)

In [14]:
def calcLocalDifferences(img,P,R):
    ld = np.zeros((img.shape[0],img.shape[1],P),dtype=np.float)

    for y in range(0, len(img)):
        for x in range(0, len(img[0])):
            center = img[y,x]
            #pixels = getNeighboringPixels(img,R,P,x,y)              
            pixels = getNeighboringPixelsPaperVersion(img,R,P,x,y)              
            ld[y,x,0:P] = pixels-center

    return ld

In [15]:
def clbp_C_Operator(img):
    c = np.mean(img)
    return np.where(img >= c, 1, 0)


In [16]:
def LDSMT(ld):
    sp = np.where(ld >= 0,1,-1)
    mp = np.absolute(ld)
    return sp, mp

In [17]:
def CLBP_S_riu2(sp,P):
    sp = np.where(sp >= 0, 1, 0)
    #sp_total = np.sum(sp,2)
    #u_value = calcTransitions(sp,P)
    #return np.where(u_value <= 2, sp_total, P+1)
    pp2 = 2**(np.array(list(range(0,P))))
    indexes = np.dot(sp,pp2)    
    return indexes

In [18]:
def CLBP_M_riu2(mp,P):
    c = np.mean(mp)
    tp = np.where(mp >= c, 1, 0)
    # pp2 = np.array([1]*P)
    # tp_total = np.dot(tp,pp2.T)
    # u_value = calcTransitions(tp,P)
    # return np.where(u_value <= 2, tp_total, P+1)
    pp2 = 2**(np.array(list(range(0,P))))
    indexes = np.dot(tp,pp2)    
    return indexes

In [19]:
def genMappingTable(P):
    mapTable = np.zeros(2**P)
    numElms = P+2
    for i in range(0,2**P):
        msb = 128 & i
        lsb = 1 & i
        count = format(i, '#010b').count('01') + format(i, '#010b').count('10') + ((msb >> (P-1)) ^ lsb)

        if count <= 2:
            mapTable[i] = bin(i).count('1')
        else:
            mapTable[i] = P+1

    return mapTable

In [20]:
def getNeighboringPixelsPaperVersion(img,R,P,x,y):
    pixels = []

    indexes = np.array(list(range(0,P)),dtype=np.float)
    dy = -R * np.sin(2 * np.pi * indexes / P)
    dx = R * np.cos(2 * np.pi * indexes / P)

    dy = np.where(abs(dy) < 5.0e-10, 0, dy)
    dx = np.where(abs(dx) < 5.0e-10, 0, dx)

    for point in range(0, P):
        r = y + dy[point]
        c = x + dx[point]

        fr = math.floor(r)
        fc = math.floor(c)

        cr = math.ceil(r)
        cc = math.ceil(c)
        
        rr = np.round(r)
        rc = np.round(c)

        if abs(c-rc) < 10e-7 and abs(r-rr) < 10e-7:
            pixels.append(get_pixel_else_0(img, int(r), int(c)))
        else:
            tr = r - fr
            tc = c - fc

            w1 = (1 - tc) * (1 - tr)
            w2 =      tc  * (1 - tr)
            w3 = (1 - tc) *      tr 
            w4 =      tc  *      tr 

            value = w1*get_pixel_else_0(img, fr, fc) + w2*get_pixel_else_0(img, fr, cc) + \
                    w3*get_pixel_else_0(img, cr, fc) + w4*get_pixel_else_0(img, cr, cc)

            pixels.append(value)

    return pixels


In [21]:
def chiSquared(p,q):
    return np.sum((p-q)**2/(p+q+1e-6))

Import dataset

In [88]:
aluminium_foil = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/aluminium_foil/sample_a/*.*')
brown_bread = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/brown_bread/sample_a/*.*')
corduroy = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/corduroy/sample_a/*.*')
cork = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/cork/sample_a/*.*')
cotton = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/cotton/sample_a/*.*')
cracker = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/cracker/sample_a/*.*')
lettuce_leaf = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/lettuce_leaf/sample_a/*.*')
linen = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/linen/sample_a/*.*')
white_bread = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/white_bread/sample_a/*.*')
wood = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/wood/sample_a/*.*')
wool = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/wool/sample_a/*.*')


# In[5]:


data = []
labels = []
j= 0
for i in aluminium_foil:
    if j>50:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(0)
    j +=1
for i in brown_bread:
    if j>100:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(1)
    j +=1
for i in corduroy:
    if j>150:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(2)
    j +=1
for i in cork:
    if j>200:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(3)
    j+=1
for i in cotton:
    if j>250 :
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(4)
    j+=1
for i in cracker:
    if j> 300:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(5)
    j+=1
for i in lettuce_leaf:
    if j> 350:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(6)
    j+=1
for i in linen:
    if j> 400:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(7)
    j+=1
for i in white_bread:
    if j> 450:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(8)
    j+=1
for i in wood:
    if j> 500:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(9)
    j+=1
for i in wool:
    if j> 550:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(10)
    j+=1

data = np.array(data)
labels = np.array(labels)

In [89]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

CLBP Github

Train dataset histogram

In [90]:
radius = 1
numPoints = 8

clbp_s = []
clbp_m = []
clbp_mc = []
clbp_s_mc = []
clbp_sm = []
clbp_smc = []

labels = []

result_y_train_clbp_s = []
result_y_train_clbp_m = []
result_y_train_clbp_mc = []
result_y_train_clbp_s_mc = []
result_y_train_clbp_sm = []
result_y_train_clbp_smc = []

nn_classifier_clbp_s = KNeighborsClassifier(n_neighbors = 1, weights='uniform', algorithm='auto', metric=chiSquared)
nn_classifier_clbp_m = KNeighborsClassifier(n_neighbors = 1, weights='uniform', algorithm='auto', metric=chiSquared)
nn_classifier_clbp_mc = KNeighborsClassifier(n_neighbors = 1, weights='uniform', algorithm='auto', metric=chiSquared)
nn_classifier_clbp_s_mc = KNeighborsClassifier(n_neighbors = 1, weights='uniform', algorithm='auto', metric=chiSquared)
nn_classifier_clbp_sm = KNeighborsClassifier(n_neighbors = 1, weights='uniform', algorithm='auto', metric=chiSquared)
nn_classifier_clbp_smc = KNeighborsClassifier(n_neighbors = 1, weights='uniform', algorithm='auto', metric=chiSquared)

mappingTable = genMappingTable(numPoints)

# train
i = 0
for imgdata in X_train:
    i+=1
    print(i)

    img_bgr = imgdata
    height, width, channel = img_bgr.shape
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img = img_gray
    # print("height,width",height,width)
    # cv2.imshow("img",img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # img = img/255
    # img = (img-np.mean(img))/np.std(img)*20+128

    dp = calcLocalDifferences(img,numPoints,radius)
    sp, mp = LDSMT(dp)

    # CLBP_C
    clbp_c = clbp_C_Operator(img)

    # CLBP_S_riu2
    clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
    mapped_clbp_s_riu2 = mappingTable[clbp_s_riu2]
    (hist_s, bins) = np.histogram(mapped_clbp_s_riu2.ravel(), bins=numPoints+2)
    clbp_s.append(hist_s.ravel())
    
    # CLBP_M_riu2
    clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
    mapped_clbp_m_riu2 = mappingTable[clbp_m_riu2]
    (hist_m, bins) = np.histogram(mapped_clbp_m_riu2.ravel(), bins=numPoints+2)
    clbp_m.append(hist_m.ravel())

    # CLBP_M/C
    hist_mc, xedges, yedges = np.histogram2d(mapped_clbp_m_riu2.ravel(), clbp_c.ravel(),bins=[numPoints+2,2])
    clbp_mc.append(hist_mc.ravel())

    # CLBP_S_M/C
    hist_s_mc = np.concatenate((hist_s,hist_mc.ravel()),axis=0)
    clbp_s_mc.append(hist_s_mc.ravel())

    # CLBP_S/M
    hist_sm, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(),mapped_clbp_m_riu2.ravel(),bins=[numPoints+2,numPoints+2])
    clbp_sm.append(hist_sm.ravel())

    # CLBP_S/M/C
    clbp_mc_sum = np.where(clbp_c > 0, clbp_m_riu2+numPoints+2, clbp_m_riu2)
    hist_smc, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(), clbp_mc_sum.ravel(),bins=[numPoints+2,2*(numPoints+2)])
    clbp_smc.append(hist_smc.ravel())


                




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


Fitting the dataset

In [91]:
# fitting
print("Fitting the models")

# model = KNeighborsClassifier(n_neighbors = 1, weights='uniform', algorithm='auto', metric=chiSquared)
# model.fit(data,y_train)

nn_classifier_clbp_s.fit(clbp_s,y_train)
nn_classifier_clbp_m.fit(clbp_m,y_train)
nn_classifier_clbp_mc.fit(clbp_mc,y_train)
nn_classifier_clbp_s_mc.fit(clbp_s_mc,y_train)
nn_classifier_clbp_sm.fit(clbp_sm,y_train)
nn_classifier_clbp_smc.fit(clbp_smc,y_train)

Fitting the models


KNeighborsClassifier(algorithm='auto', leaf_size=30,
                     metric=<function chiSquared at 0x7f642db76440>,
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

Test dataset histogram


In [92]:
# test
print("Test steps:")
i = 0
for imgdata in X_test:
    print(i)
    # image_file = '/content/00000'+str(i)+'.bmp'
    img_bgr = imgdata
    height, width, channel = img_bgr.shape
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img = img_gray
    dp = calcLocalDifferences(img,numPoints,radius)
    sp, mp = LDSMT(dp)

    # CLBP_C
    clbp_c = clbp_C_Operator(img)

    # CLBP_S_riu2
    clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
    mapped_clbp_s_riu2 = mappingTable[clbp_s_riu2]
    (hist_s, bins) = np.histogram(mapped_clbp_s_riu2.ravel(), bins=numPoints+2)
    result_labels_clbp_s.append((y_test[i],
        nn_classifier_clbp_s.predict([hist_s.ravel()])))
    
    # CLBP_M_riu2
    clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
    mapped_clbp_m_riu2 = mappingTable[clbp_m_riu2]
    (hist_m, bins) = np.histogram(mapped_clbp_m_riu2.ravel(), bins=numPoints+2)
    result_labels_clbp_m.append((y_test[i],
        nn_classifier_clbp_m.predict([hist_m.ravel()])))


    # CLBP_M/C
    hist_mc, xedges, yedges = np.histogram2d(mapped_clbp_m_riu2.ravel(), clbp_c.ravel(),bins=[numPoints+2,2])
    result_labels_clbp_mc.append((y_test[i],
        nn_classifier_clbp_mc.predict([hist_mc.ravel()])))

    # CLBP_S_M/C
    hist_s_mc = np.concatenate((hist_s,hist_mc.ravel()),axis=0)
    result_labels_clbp_s_mc.append((y_test[i],
        nn_classifier_clbp_s_mc.predict([hist_s_mc.ravel()])))

    # CLBP_S/M
    hist_sm, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(),mapped_clbp_m_riu2.ravel(),bins=[numPoints+2,numPoints+2])
    result_labels_clbp_sm.append((y_test[i],
        nn_classifier_clbp_sm.predict([hist_sm.ravel()])))

    # CLBP_S/M/C
    clbp_mc_sum = np.where(clbp_c > 0, clbp_m_riu2+numPoints+2, clbp_m_riu2)
    hist_smc, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(), clbp_mc_sum.ravel(),bins=[numPoints+2,2*(numPoints+2)])
    result_labels_clbp_smc.append((y_test[i],
        nn_classifier_clbp_smc.predict([hist_smc.ravel()])))
    
    i+=1

Test steps:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110


Only combined MS histogram

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(hist_clbp_ms, y_train)
cv2_imshow(X_test[4])
x_test_12 = cv2.cvtColor(X_test[4], cv2.COLOR_BGR2GRAY)
cv2_imshow(x_test_12)
print(neigh.predict([hist_clbp_ms_test[4]]))
acc_ms = neigh.score(hist_clbp_ms_test, y_test)
# print(neigh.summary())
print("[INFO] histogram accuracy: {:.2f}%".format(acc_ms * 100))

only S histogram

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(clbp_s, y_train)
cv2_imshow(X_test[4])
x_test_12 = cv2.cvtColor(X_test[4], cv2.COLOR_BGR2GRAY)
cv2_imshow(x_test_12)

print(neigh.predict([clbp_s_test[4]]))
acc_s = neigh.score(clbp_s_test, y_test)
# print(neigh.summary())
print("[INFO] histogram accuracy: {:.2f}%".format(acc_s * 100))

Predictions results for CLBP_S_riu2

In [93]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s[j][0] == result_labels_clbp_s[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 94
accuracy = 84.68468468468468


Predictions results for CLBP_M_riu2

In [94]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_m[j][0] == result_labels_clbp_m[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 92
accuracy = 82.88288288288288


Predictions results for CLBP_M/C

In [95]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_mc[j][0] == result_labels_clbp_mc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 99
accuracy = 89.1891891891892


Predictions results for CLBP_S_M/C

In [96]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s_mc[j][0] == result_labels_clbp_s_mc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 100
accuracy = 90.09009009009009


Predictions results for CLBP_S/M

In [97]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_sm[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 102
accuracy = 91.8918918918919


Predictions results for CLBP_S/M/C

In [98]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_smc[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 102
accuracy = 91.8918918918919


Plotting decision boundaries of KNN classifier